[View in Colaboratory](https://colab.research.google.com/github/Stallon-niranjan/Inkers_EIP_WDocs/blob/master/NIRANJAN_ANANDAKUMAR_BATCH_2_ASSIGNMENT4B.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 200
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [23]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 12)   48          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 12)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

Total params: 118,918
Trainable params: 114,394
Non-trainable params: 4,524
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [32]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 135s 3ms/step - loss: 0.8207 - acc: 0.7058 - val_loss: 0.9048 - val_acc: 0.6975
Epoch 2/50
44600/50000 [=========================>....] - ETA: 13s - loss: 0.7999 - acc: 0.7137

50000/50000 [==============================] - 133s 3ms/step - loss: 0.7993 - acc: 0.7140 - val_loss: 0.9097 - val_acc: 0.6944
Epoch 3/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.7820 - acc: 0.7186 - val_loss: 0.9751 - val_acc: 0.6828
Epoch 4/50
23000/50000 [============>.................] - ETA: 1:07 - loss: 0.7661 - acc: 0.7264

50000/50000 [==============================] - 133s 3ms/step - loss: 0.7686 - acc: 0.7256 - val_loss: 0.9824 - val_acc: 0.6796
Epoch 5/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.7501 - acc: 0.7306 - val_loss: 0.9513 - val_acc: 0.6854
Epoch 6/50
17800/50000 [=========>....................] - ETA: 1:20 - loss: 0.7242 - acc: 0.7408

50000/50000 [==============================] - 133s 3ms/step - loss: 0.7322 - acc: 0.7398 - val_loss: 0.7992 - val_acc: 0.7324
Epoch 7/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.7176 - acc: 0.7463 - val_loss: 0.9348 - val_acc: 0.7019
Epoch 8/50
16600/50000 [========>.....................] - ETA: 1:23 - loss: 0.6871 - acc: 0.7552

50000/50000 [==============================] - 133s 3ms/step - loss: 0.7070 - acc: 0.7481 - val_loss: 0.8229 - val_acc: 0.7252
Epoch 9/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.7022 - acc: 0.7498 - val_loss: 0.8338 - val_acc: 0.7257
Epoch 10/50
16400/50000 [========>.....................] - ETA: 1:24 - loss: 0.6839 - acc: 0.7543

...

50000/50000 [==============================] - 133s 3ms/step - loss: 0.5069 - acc: 0.8222 - val_loss: 0.7283 - val_acc: 0.7744
Epoch 43/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.5062 - acc: 0.8205 - val_loss: 0.6684 - val_acc: 0.7883
Epoch 44/50
16000/50000 [========>.....................] - ETA: 1:24 - loss: 0.4940 - acc: 0.8228

50000/50000 [==============================] - 133s 3ms/step - loss: 0.4964 - acc: 0.8231 - val_loss: 0.7324 - val_acc: 0.7797
Epoch 45/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.5004 - acc: 0.8243 - val_loss: 0.6533 - val_acc: 0.7956
Epoch 46/50
16000/50000 [========>.....................] - ETA: 1:24 - loss: 0.4955 - acc: 0.8237

50000/50000 [==============================] - 132s 3ms/step - loss: 0.4955 - acc: 0.8235 - val_loss: 0.9213 - val_acc: 0.7266
Epoch 47/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.4954 - acc: 0.8248 - val_loss: 0.6772 - val_acc: 0.7891
Epoch 48/50
16000/50000 [========>.....................] - ETA: 1:25 - loss: 0.4773 - acc: 0.8289

50000/50000 [==============================] - 133s 3ms/step - loss: 0.4880 - acc: 0.8260 - val_loss: 0.7389 - val_acc: 0.7706
Epoch 49/50
50000/50000 [==============================] - 133s 3ms/step - loss: 0.4888 - acc: 0.8270 - val_loss: 0.8049 - val_acc: 0.7709
Epoch 50/50
16000/50000 [========>.....................] - ETA: 1:25 - loss: 0.4818 - acc: 0.8264

50000/50000 [==============================] - 133s 3ms/step - loss: 0.4886 - acc: 0.8272 - val_loss: 0.7611 - val_acc: 0.7698


In [33]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7611301194190979
Test accuracy: 0.7698


In [34]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')